# EDA (Exploratory Data Analysis)

### Loading the data

In [ ]:
from ucimlrepo import fetch_ucirepo
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV  
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, AdaBoostClassifier, GradientBoostingClassifier, HistGradientBoostingClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error, r2_score, precision_score, recall_score, f1_score, classification_report
from sklearn.utils.class_weight import compute_class_weight, compute_sample_weight
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
import numpy as np
import pandas as pd
import os

In [ ]:
# fetch dataset
wine_quality = fetch_ucirepo(id=186)
# data (as pandas dataframes)
X = wine_quality.data.features
y = wine_quality.data.targets

In [ ]:
df = X.copy()
df['quality'] = y

### Exploratory Data Analysis (EDA)

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.isna().sum()

In [ ]:
df.duplicated().sum()

As part of the data preprocessing, we will remove **missing values (null entries)** to prevent feeding erroneous or incomplete information into the system, which could compromise the reliability of the results.  
Additionally, **duplicate entries** will be identified and eliminated to avoid data redundancy, improve computational efficiency, and reduce the risk of overfitting during the training phase of any predictive models.

In [ ]:
df.isna().sum()

In [ ]:
df.duplicated().sum()

In [ ]:
df.drop_duplicates(inplace=True)
df.shape

We visualize the distribution of the data for each column of the dataset. To do this, we will use boxplot, that is a method for demonstrating graphically the locality, spread and skewness groups of numerical data through their quartiles.

In [ ]:
# Boxplot de cada columna
plt.figure(figsize=(20, 10))

for i, col in enumerate(df.columns):
    plt.subplot(3, 4, i + 1)
    sns.boxplot(y=df[col])
    plt.title(col)

Next, **outliers are removed to enhance data quality and reliability**. Outliers can distort statistical calculations, cause models to overfit, and reduce overall accuracy. By applying the IQR method, we retain only the data points that fall within a reasonable range, improving the dataset for subsequent analysis or modeling.

Outliers affect each model in a different way. For example, the following explains the effects of training some machine learning models with outliers:

- **`Linear Regression`**: They can affect the estimation of the model coefficients, which can lead to inaccurate predictions. This is because the estimation of these coefficientes is done by minimizing the sum of the quadratic errors, and outliers can generate large quadratic errors, which affects model performance.

- **`Random Forest Regressor`**: It is more robust to outliers. This is because this model consists of making predictions from several decision trees from subsets of the data and then calculation the mean of each of the results obtained. In this way, the outliers lose weight against other data with a hihger degree of similarity to te data used.

- **`Stochastic Gradient Descent`**: Outliers can negatively affect this model. This is because, since the training algorithm updates the parameters iteratively using only one training example at a time, outliers can cause very large parameter updates in the learning process preventing it from converging effectively.

- **`Random Forest Classifier`**: The degree to which outliers affect is the same as the *Random Forest Regressor*, since the only difference between the two models is the output they generate.

- **`Logistic Regression`**: This model is sensitive to outliers as they affect the estimation of coefficients. However, the versions with regularization (Lasso and Ridge) penalize large coefficients, which reduces the influence of outliers, thus improving the performance of the model.

- **`K-Nearest Neighbors`**:It is very sensitive to outliers, since this model is based on the idea that nearby data points have similar characteristics (*"If it walks like a duck, quacks like a duck, and looks like a duck, then it's probably a duck. ”*). If an outlier is close to a decision point, it may cause the model to classify incorrectly.

- **`Naive Bayes`**: This is relatively robust to outliers. This model assumes that the features follow a normal distribution and calculates probabilities based only on the mean and standard deviation. The outliers, having extremely low probability, have minimal impact on the final predictions, as their influence is diluted in the overall calculation.

In [ ]:
X = df.drop('quality', axis=1)
y = df['quality']

# Eliminar los outliers de X
Q1 = X.quantile(0.25)
Q3 = X.quantile(0.75)
IQR = Q3 - Q1

X = X[~((X < (Q1 - 1.5 * IQR)) | (X > (Q3 + 1.5 * IQR))).any(axis=1)]

# Eliminar de y las filas que se eliminaron de X
y = y[y.index.isin(X.index)]

df = X.copy()
df['quality'] = y

In [ ]:
df.shape

Once deleted the outliers, the boxplot are displayed for comparison

In [ ]:
# Boxplot de cada columna una vez eliminados los outliers
plt.figure(figsize=(20, 10))

for i, col in enumerate(df.columns):
    plt.subplot(3, 4, i + 1)
    sns.boxplot(y=df[col])
    plt.title(col)

The distributions of each of the columns of the dataframe are as follows

In [ ]:
df.hist(bins=20, figsize=(20, 20))
plt.show()

Following, we analyze the distribution of the instances of each class. In this way, we will be able to check if the classes are unbalanced.

In [ ]:
sns.countplot(x=df['quality'])
plt.title('Quality Count')
plt.xlabel('Quality Value')
plt.ylabel('Count')
plt.show()

print(df['quality'].value_counts())

The dataset reveals that most instances reflect quality values ranging from 5 to 7. This observation indicates that the majority of the wines evaluated fall within a medium-quality range, suggesting a concentration of average performance rather than extreme highs or lows in quality.

For a better understanding of data distribution, we will calculate the skewness value, that is an indicator of the degree of asymmetry of the data.

In [ ]:
# Plot de skewness 
skewness = df.skew()
skewness.plot(kind='bar', title='Skewness of the numerical features', figsize=(10, 5))
plt.show()

Once the results are obtained, it can be concluded that the `density` or `citric_acid` columns have almost perfect symmetry, while columns such as `chlorides` or `volatile_acidity` have a significant skew to the right.

Some models assume normality in the data, so skewed distributions can lead to less accurate models and inadequate predictions. To address this, a log normalization transformation will be used to reduce positive skewness and compress high values. Skewness values greater than ±0.8 are considered skewed distributions, so we will apply normalization to these features.

In [ ]:
# Normalización de las columnas con skewness mayor a +-0.8
skewness = skewness[abs(skewness) > 0.8]
skewed_features = skewness.index
for feature in skewed_features:
    df[feature] = np.log1p(df[feature])
    print(feature)

Check if skewness value of each column has decreased

In [ ]:
# Plot de skewness 
skewness = df.skew()
skewness.plot(kind='bar', title='Skewness of the numerical features', figsize=(10, 5))
plt.show()

The `chlorides` feature still has a very high skewness, so it would be advisable to remove it when training the different models.

In [ ]:
# Pairplot de cada columna del dataframe
sns.pairplot(df, diag_kind='kde')
plt.show()

To observe the correlation between the different columns of the dataframe, we use the correlation matrix. Depending on the range of the values, we can interpret the following relationships:

- **Perfect negative correlation**: The range is [-1, -1]. This indicates that the variables are completely inversely related.

- **Strong negative correlation**: The range is (-1, -0.7]. This indicates a strong inverse relationship: one variable tends to decrease as the other increases.

- **Moderate negative correlation**: The range is (-0.7, -0.3]. This indicates a moderate inverse relationship between the variables.

- **Weak negative correlation**: The range is (-0.3, 0). This indicates a weak inverse relationship, but some tendency may exist.

- **No correlation**: The range is [0, 0]. This indicates that there is no linear relationship between the variables.

- **Weak positive correlation**: The range is (0, 0.3). This indicates a weak direct relationship, but some tendency may exist.

- **Moderate positive correlation**: The range is [0.3, 0.7). This indicates a moderate direct relationship between the variables.

- **Strong positive correlation**: The range is [0.7, 1). This indicates a strong direct relationship: one variable tends to increase as the other does.

- **Perfect positive correlation**: The range is [1, 1]. This indicates that the variables are completely directly related.

In [ ]:
corr_matrix = df.corr()
plt.figure(figsize=(15, 10))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')
plt.title('Correlation Matrix')
plt.show()

Finally, this code trains a **Random Forest Classifier** to analyze the importance of features in predicting the target variable `quality`. It calculates the contribution of each feature using the model's `feature_importances_` attribute, sorts the features by importance, and visualizes the results in a bar chart. This helps identify which features have the most significant impact on the model's predictions.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
import pandas as pd

X = df.drop('quality', axis=1)
y = df['quality']

# Suponiendo que tienes un dataframe X con las características y un vector y con las etiquetas
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X, y)

# Obtener la importancia de las características
importances = model.feature_importances_

# Crear un DataFrame con los resultados
importance_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': importances
})

# Ordenar por importancia
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# Visualizar la importancia de las características
plt.figure(figsize=(10, 6))
plt.barh(importance_df['Feature'], importance_df['Importance'])
plt.title('Importancia de las Características')
plt.xlabel('Importancia')
plt.ylabel('Características')
plt.show()

# Ver la lista de importancias
print(importance_df)


The feature importance analysis highlights alcohol as the most significant predictor of wine quality, which aligns with the common understanding that higher alcohol content often correlates with perceived quality in wines. However, density and volatile acidity, which also ranked highly, are typically indicators of wine's physical and chemical balance rather than direct quality markers.  
Interestingly, attributes like chlorides and sulfur dioxide levels, which are more commonly associated with wine preservation than quality, show notable influence, suggesting that stability and preservation may indirectly impact quality perception.

# Classification

## Random Forest

There are quite high correlations in the correlation matrix.
Considering the importance of each of the features, it was decided to eliminate the columns "density" and "free_sulfur_dioxide".
In addition, the features "sulphates", "residual_sugar" and "pH" have a very low correlation with wine quality.

We will eliminate the correlated and less important features.
Sulphates” will also be eliminated as it has little importance and correlation with wine quality.


In [ ]:
rf_df = df.drop(['density', 'free_sulfur_dioxide', 'sulphates'], axis=1)

In addition, in general, we will divide the dataset into train and test.
This will be done to evaluate the training of the model.
We will try to keep the proportion of classes the same in both sets.

This is why stratify is used.


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(rf_df.drop('quality', axis=1), rf_df['quality'], test_size=0.2,stratify=rf_df['quality'],random_state=42)

In each algorithm there are certain hyperparameters that can be adjusted to improve the performance of the model.
To find those that improve the performance of the model, several combinations will be tested.
Checking which of them maximizes certain metrics, such as accuracy, recall or f1-score.


In this case, the random forest has the following hyperparameters:

- `n_estimators`: number of trees in the forest.
- `max_depth`: maximum depth of the trees
- `min_samples_split`: minimum number of samples needed to split one node
- `min_samples_leaf`: minimum number of samples needed in a leaf node


In [ ]:
param_grid = {
    'n_estimators': [300, 500],
    'max_depth': [10, 20, 30, 40, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
}

# class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
# class_weights_dict = dict(zip(np.unique(y_train), class_weights))

rf = RandomForestClassifier(random_state=42)

grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, n_jobs=-1)
grid_search.fit(X_train, y_train)

print(grid_search.best_params_)
print(grid_search.best_estimator_)
print(grid_search.best_score_)

Once the best hyperparameters have been obtained, the model will be trained with the training dataset and evaluated with the test dataset.
The metrics obtained will be displayed and compared with those obtained in the training.
We will use the confusion matrix to see how the model behaves in each class.


In [ ]:
# Modelo con los mejores hiperparámetros
best_rf = grid_search.best_estimator_
y_pred = best_rf.predict(X_test)

# Calcular la precisión
accuracy = accuracy_score(y_test, y_pred)
print('Precisión:', accuracy)

In [ ]:
# Matriz de confusión
rf_conf_matrix = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(10, 6))
sns.heatmap(rf_conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=np.unique(y_test), yticklabels=np.unique(y_test))
plt.title('Matriz de Confusión')
plt.xlabel('Predicción')
plt.ylabel('Real')
plt.show()

## Logistic Regression


In [ ]:
lr_df = df.drop(['density', 'free_sulfur_dioxide', 'sulphates'], axis=1)

In [ ]:
X = lr_df.drop('quality', axis=1)
y = lr_df['quality']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

Normalization of the data is important so that the model converges faster and so that there are no features that have more weight than others, since it brings them to a common scale.
Especially when we are working with models that use Euclidean distance, such as logistic regression.


In [ ]:
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(X_train)
x_test_scaled = scaler.transform(X_test)

### Lasso

For this model, the hyperparameter that can be set is `C`, which is the inverse of the regularization strength.

The difference between Lasso and Ridge is that Lasso can lead to some coefficients being 0, which can be useful for feature selection.

In [ ]:
param_grid = {
    'C': list(map(lambda x: x/100, range(1, 200, 1))),
}

lasso_model = LogisticRegression(penalty='l1', solver='saga')

grid_search = GridSearchCV(estimator=lasso_model, param_grid=param_grid, cv=3, n_jobs=-1, scoring="f1_macro")
grid_search.fit(X_train, y_train)


In [ ]:
lasso_model = grid_search.best_estimator_
lasso_model.fit(x_train_scaled, y_train)
y_pred_lasso = lasso_model.predict(x_test_scaled)
print('Precisión Lasso:', accuracy_score(y_test, y_pred_lasso))


In [ ]:
lasso_conf_matrix = confusion_matrix(y_test, y_pred_lasso)
sns.heatmap(lasso_conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=np.unique(y_test), yticklabels=np.unique(y_test))
plt.title('Matriz de Confusión Lasso')
plt.xlabel('Predicción')
plt.ylabel('Real')
plt.show()

### Ridge

Ridge is similar to Lasso, but does not lead to the coefficients being 0.
This can be useful if we do not want to eliminate features and instead want to take them all into account even if some are less important.

In [ ]:
param_grid = {
    'C': list(map(lambda x: x/100, range(1, 200, 1))),
}

ridge_model = LogisticRegression(penalty='l2')

grid_search = GridSearchCV(estimator=ridge_model, param_grid=param_grid, cv=3, n_jobs=-1, scoring='accuracy')
grid_search.fit(X_train, y_train)


In [ ]:
print(grid_search.best_params_)


In [ ]:
ridge_model = grid_search.best_estimator_

ridge_model.fit(x_train_scaled, y_train)
y_pred_ridge = ridge_model.predict(x_test_scaled)
print('Precisión Ridge:', accuracy_score(y_test, y_pred_ridge))


In [ ]:
ridge_conf_matrix = confusion_matrix(y_test, y_pred_ridge)
sns.heatmap(ridge_conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=np.unique(y_test), yticklabels=np.unique(y_test))
plt.title('Matriz de Confusión Ridge')
plt.xlabel('Predicción')
plt.ylabel('Real')
plt.show()

## Ensemble


The following models will be ensembled: Random Forest, Linear Regression (Lasso) and Linear Regression (Ridge).

It is useful because it is possible to combine models that have different strengths and weaknesses.
Thus, a more robust and generalizable model can be obtained, which does not depend so much on the characteristics of a single model, which will give us a better overall performance.


In [ ]:
ensemble_df = df.drop(['density', 'free_sulfur_dioxide', 'sulphates'], axis=1)

In [ ]:
X = ensemble_df.drop('quality', axis=1)
y = ensemble_df['quality']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

One possibility is to use a voting model, which consists of each model voting for the class it believes to be the correct one, and the class with the most votes is chosen.
To this a weight can be added to each model, so that they do not all have the same weight in the final decision.
In our case, the same weight will be given to each model, since we will only use 3 models and increasing the weight of one of them may lead to an overfitting to that model.


In [ ]:
from sklearn.ensemble import VotingClassifier

voting_model = VotingClassifier(estimators=[('lasso', lasso_model), ('ridge', ridge_model), ('random_forest', best_rf)], voting='soft')
voting_model.fit(X_train, y_train)

y_pred_voting = voting_model.predict(X_test)
print('Precisión Voting:', accuracy_score(y_test, y_pred_voting))

Another possibility is to use a stacking model, which consists of a model being trained with the predictions of the other models and the original features.
In this way, the final model can learn to combine the predictions of the other models in a more optimal way.
In our case, a logistic regression model will be used as the final model, since it is a simple and fast model to train.
A more complex model could also be used and a hyperparameter search could be performed to find those that maximize the performance of the final model.


In [ ]:
from sklearn.ensemble import StackingClassifier

stacking_model = StackingClassifier(estimators=[('lasso', lasso_model), ('ridge', ridge_model), ('random_forest', best_rf)], final_estimator=LogisticRegression())
stacking_model.fit(X_train, y_train)

y_pred_stacking = stacking_model.predict(X_test)
print('Precisión Stacking:', accuracy_score(y_test, y_pred_stacking))

## Boosting

Boosting can help us to better classify classes that are unbalanced, which occurs in our dataset.
In addition, it can help improve model performance, since several models are trained sequentially and each one is trained to correct the errors of the previous one.


For our problem, we will try AdaBoosting, GradientBoosting and HistGradientBoosting.
The differences between them are the following:

- AdaBoost: trains several models sequentially, each one is trained to correct the errors of the previous one.
- GradientBoosting: trains several models sequentially, each one is trained to correct the errors of the previous one, but in this case a decision tree is trained in each iteration.
- HistGradientBoosting: similar to GradientBoosting, but in this case a histogram is used to speed up training.

Each of these models has hyperparameters that can be adjusted to improve model performance.
For example, the number of estimators in addition to the hyperparameters of the decision trees.

Hyperparameter searches could be done to find those that maximize model performance.
This will not be done since we have seen how it would be done in the [random forest](#Random-Forest) section.


In [ ]:
boosting_df = df.drop(['density', 'free_sulfur_dioxide', 'sulphates'], axis=1)

In [ ]:
X = boosting_df.drop('quality', axis=1)
y = boosting_df['quality']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [ ]:
rf_base = RandomForestClassifier(n_estimators=100, random_state=42)
ada_boost = AdaBoostClassifier(estimator=rf_base, n_estimators=50, random_state=42)
gradient_boost = GradientBoostingClassifier(n_estimators=45, random_state=42)
hist_gradient_boost = HistGradientBoostingClassifier(max_iter=15, random_state=42)

In [ ]:
ada_boost.fit(X_train, y_train)
gradient_boost.fit(X_train, y_train)
hist_gradient_boost.fit(X_train, y_train)

In [ ]:
y_pred_ada = ada_boost.predict(X_test)
y_pred_gradient = gradient_boost.predict(X_test)
y_pred_hist_gradient = hist_gradient_boost.predict(X_test)

In [ ]:
accuracy_ada = accuracy_score(y_test, y_pred_ada)
print('Precisión AdaBoost:', accuracy_ada)

accuracy_gradient = accuracy_score(y_test, y_pred_gradient)
print('Precisión GradientBoost:', accuracy_gradient)

accuracy_hist_gradient = accuracy_score(y_test, y_pred_hist_gradient)
print('Precisión HistGradientBoost:', accuracy_hist_gradient)

It should be noted that, although a hyperparameter search was not performed, they gave results similar to those obtained in the [random forest](#Random-Forest) section.
Probably, if a hyperparameter search were done, better results could be obtained.


In [ ]:
knn = KNeighborsClassifier(n_neighbors=5)

# Train the model
knn.fit(X_train, y_train)

# Predict on the test set
y_pred = knn.predict(X_test)

# Evaluate the model
accuracy = knn.score(X_test, y_test)
print(f'Accuracy: {accuracy}')

## K-Nearest Neighbors


It is a supervised learning algorithm that can be used for both classification and regression.
In case of classification, the class that is most repeated among the k nearest neighbors is assigned, while in regression the mean of the k nearest neighbors is assigned.


It has only 3 hyperparameters:

- `n_neighbors`: number of nearest neighbors
- `weights`: weight given to nearest neighbors
- `metric`: metric used to calculate the distance between instances


It is affected by irrelevant variables and by the scale of the variables.
Both have already been discussed in the previous sections.


In [ ]:
knn_df = df.drop(['density', 'free_sulfur_dioxide', 'sulphates'], axis=1)

In [ ]:
X = knn_df.drop('quality', axis=1)
y = knn_df['quality']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [ ]:
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(X_train)
x_test_scaled = scaler.transform(X_test)

Using classification:


In [ ]:
param_grid = {
    'n_neighbors': list(range(1, 20, 1)),
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'minkowski'],
}

knn = KNeighborsClassifier()
grid_search = GridSearchCV(estimator=knn, param_grid=param_grid, cv=3, n_jobs=-1)

grid_search.fit(X_train, y_train)

print(grid_search.best_params_)
print(grid_search.best_estimator_)
print(grid_search.best_score_)


In [ ]:
# Modelo con los mejores hiperparámetros
best_knn = grid_search.best_estimator_
y_pred = best_knn.predict(X_test)

# Calcular la precisión
accuracy = accuracy_score(y_test, y_pred)
print('Precisión:', accuracy)

In [ ]:
# Matriz de confusión
knn_conf_matrix = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(10, 6))
sns.heatmap(knn_conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=np.unique(y_test), yticklabels=np.unique(y_test))
plt.title('Matriz de Confusión')
plt.xlabel('Predicción')
plt.ylabel('Real')
plt.show()

Using regression:


In [ ]:
param_grid = {
    'n_neighbors': list(range(1, 20, 1)),
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'minkowski'],
}

knn = KNeighborsRegressor()
grid_search = GridSearchCV(estimator=knn, param_grid=param_grid, cv=3, n_jobs=-1, scoring='neg_mean_squared_error')

grid_search.fit(X_train, y_train)

print(grid_search.best_params_)
print(grid_search.best_estimator_)
print(grid_search.best_score_)


In [ ]:
# Modelo con los mejores hiperparámetros
best_knn = grid_search.best_estimator_
y_pred = best_knn.predict(X_test)

# Calcular el error
mse = mean_squared_error(y_test, y_pred)

# Calcular el coeficiente de determinación R^2
r2 = r2_score(y_test, y_pred)

# Mostrar los resultados
print(f"Error cuadrático medio (MSE): {mse}")
print(f"Coeficiente de determinación (R^2): {r2}")
print(f"Precisión: {accuracy_score(y_test, np.round(y_pred))}")

To visualize the results, the confusion matrix has been used by rounding the predictions to the nearest class.


In [ ]:
y_pred_round = np.round(y_pred)

In [ ]:
# Calcular la precisión
accuracy = accuracy_score(y_test, y_pred_round)
print('Precisión:', accuracy)


In [ ]:
# Matriz de confusión
rf_conf_matrix = confusion_matrix(y_test, y_pred_round)

plt.figure(figsize=(10, 6))
sns.heatmap(rf_conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=np.unique(y_test), yticklabels=np.unique(y_test))
plt.title('Matriz de Confusión')
plt.xlabel('Predicción')
plt.ylabel('Real')
plt.show()

# Regression

This problem is a classification problem, but it can also be posed as a regression problem.
In this case, wine quality will be used as the target variable and the other characteristics as predictor variables.

This is possible since wine quality is an ordinal variable, i.e., it has an order.
So it can be posed as a regression problem, where an attempt is made to predict a numerical value rather than a class.


## Random Forest Regressor


For this one, we will do the same process as in classification, but in this case we will use a Random Forest Regressor instead of a Random Forest Classifier.

We will also use `compute_sample_weight` so that the model takes into account the imbalance of the classes.
This causes those instances of minority classes to have more weight in the training of the model.


In [ ]:
rf_df = df.drop(['density', 'free_sulfur_dioxide', 'sulphates'], axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(rf_df.drop('quality', axis=1), rf_df['quality'], test_size=0.2,stratify=rf_df['quality'],random_state=42)

In [ ]:
param_grid = {
    'n_estimators': [300, 500],
    'max_depth': [10, 20, 30, 40, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
}
sample_weights = compute_sample_weight(class_weight='balanced', y=y_train)

# class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
# class_weights_dict = dict(zip(np.unique(y_train), class_weights))

rf_regressor = RandomForestRegressor(random_state=42)

grid_search = GridSearchCV(estimator=rf_regressor, param_grid=param_grid, cv=3, n_jobs=-1)
grid_search.fit(X_train, y_train, sample_weight=sample_weights)

print(grid_search.best_params_)
print(grid_search.best_estimator_)
print(grid_search.best_score_)

In [ ]:
# Modelo con los mejores hiperparámetros
best_rf = grid_search.best_estimator_
y_pred = best_rf.predict(X_test)

In [ ]:
# Calcular el error cuadrático medio (MSE)
mse = mean_squared_error(y_test, y_pred)

# Calcular el coeficiente de determinación R^2
r2 = r2_score(y_test, y_pred)

# Mostrar los resultados
print(f"Error cuadrático medio (MSE): {mse}")
print(f"Coeficiente de determinación (R^2): {r2}")
print(f"Precisión: {accuracy_score(y_test, np.round(y_pred))}")

In [ ]:
y_pred_round = np.round(y_pred)

In [ ]:
# Matriz de confusión
rf_conf_matrix = confusion_matrix(y_test, y_pred_round)

plt.figure(figsize=(10, 6))
sns.heatmap(rf_conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=np.unique(y_test), yticklabels=np.unique(y_test))
plt.title('Matriz de Confusión')
plt.xlabel('Predicción')
plt.ylabel('Real')
plt.show()

In [ ]:
accuracy = accuracy_score(y_test, y_pred_round)
print("Precisión:", accuracy)

## Multiple Regression

We will test with multiple linear regression, which is a model that attempts to predict a numerical variable from several predictor variables.
The model attempts to find the linear relationship between the predictor variables and the target variable.
It assumes linearity, independence of the predictor variables and normality of the errors, which is the main problem of this model.


In [ ]:
lr_df = df.drop(['density', 'free_sulfur_dioxide', 'sulphates'], axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(lr_df.drop('quality', axis=1), lr_df['quality'], test_size=0.2,stratify=rf_df['quality'],random_state=42)

In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)

In [ ]:
y_pred = lr.predict(X_test)

In [ ]:
# Calcular el error cuadrático medio (MSE)
mse = mean_squared_error(y_test, y_pred)

# Calcular el coeficiente de determinación R^2
r2 = r2_score(y_test, y_pred)

# Mostrar los resultados
print(f"Error cuadrático medio (MSE): {mse}")
print(f"Coeficiente de determinación (R^2): {r2}")
print(f"Precisión: {accuracy_score(y_test, np.round(y_pred))}")


## Stochastic Gradient Descent (SGD)

It needs the variables to be normalized, so the data will be normalized before training the model.
It is good in case of having a large number of instances, since it is trained iteratively and does not need to load all the data in memory.
For our case, it is not so useful since the dataset is not very large.


In [ ]:
sgd_df = df.drop(['density', 'free_sulfur_dioxide', 'sulphates'], axis=1)

In [ ]:
X = sgd_df.drop('quality', axis=1)
y = sgd_df['quality']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [ ]:
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(X_train)
x_test_scaled = scaler.transform(X_test)

In [ ]:
from sklearn.linear_model import SGDRegressor

sgd = SGDRegressor(max_iter=1000, loss='squared_error', random_state=42)

In [ ]:
sgd.fit(x_train_scaled, y_train)
y_pred = sgd.predict(x_test_scaled)

In [ ]:
# Calcular el error cuadrático medio (MSE)
mse = mean_squared_error(y_test, y_pred)

# Calcular el coeficiente de determinación R^2
r2 = r2_score(y_test, y_pred)

# Mostrar los resultados
print(f"Error cuadrático medio (MSE): {mse}")
print(f"Coeficiente de determinación (R^2): {r2}")
print(f"Precisión: {accuracy_score(y_test, np.round(y_pred))}")


# Try best features combination

Here we will try to test all possible combinations of features to see which one gives us the best result.
For this, we will use a Random Forest Classifier and a Logistic Regression.
In addition to each combination of features, we will try several combinations of hyperparameters to find those that maximize the performance of the model.

This is an attempt to solve the brute force classification problem, since testing all possible combinations of features is computationally very expensive.


Here we will define functions to do the hyperparameter search, one function for the random forest and one for the logistic regression.
Each of them will return the accuracy and other metrics for the given combination of features.

In [ ]:
def classifier_algorithm_rf(X,y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,stratify=y,random_state=42)

    param_grid = {
        'n_estimators': [300, 500],
        'max_depth': [20, 30, 40],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [2, 4],
    }

    rf = RandomForestClassifier(random_state=42)

    grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, n_jobs=-1)
    grid_search.fit(X_train, y_train)

    best_rf = grid_search.best_estimator_
    y_pred = best_rf.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    return accuracy, f1, precision, recall

def classifier_algorithm_lr(X,y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,stratify=y,random_state=42)

    scaler = StandardScaler()
    x_train_scaled = scaler.fit_transform(X_train)
    x_test_scaled = scaler.transform(X_test)

    lasso_model = LogisticRegression(penalty='l1', solver='saga', C=0.1)
    lasso_model.fit(x_train_scaled, y_train)
    y_pred_lasso = lasso_model.predict(x_test_scaled)

    accuracy = accuracy_score(y_test, y_pred_lasso)
    f1 = f1_score(y_test, y_pred_lasso, average='weighted')
    precision = precision_score(y_test, y_pred_lasso, average='weighted')
    recall = recall_score(y_test, y_pred_lasso, average='weighted')
    return accuracy, f1, precision, recall

Now, we will create all combinations of at least 4 features:


In [ ]:
import itertools

def combinacines_df(df):
    combinaciones = []
    for i in range(4, len(df.columns)+1):
        combinaciones.extend(list(itertools.combinations(df.columns, i)))
    return combinaciones

We will also create a function to save the results in a csv file, to be able to compare them more easily.


In [ ]:
def guardar_en_csv(combinacion,accuracy, f1, precision, recall,file_path):
    if not os.path.exists(file_path):
        with open(file_path, 'w') as f:
            f.write('combinacion,accuracy,f1,precision,recall\n')
    with open(file_path, 'a') as f:
        f.write(f'{combinacion},{accuracy},{f1},{precision},{recall}\n')

These loops will take care of testing all combinations and saving them.

In [ ]:
combinaciones = combinacines_df(df.drop('quality', axis=1))

import warnings
warnings.filterwarnings('ignore')  # Ignora todos los warnings

for combinacion in combinaciones:
    X = df[list(combinacion)]
    y = df['quality']
    accuracy_lr, f1_lr, precision_lr, recall_lr = classifier_algorithm_lr(X, y)
    guardar_en_csv(combinacion,accuracy_lr, f1_lr, precision_lr, recall_lr,'resultados_lr.csv')

for combinacion in combinaciones:
    X = df[list(combinacion)]
    y = df['quality']
    accuracy_rf, f1_rf, precision_rf, recall_rf = classifier_algorithm_rf(X, y)
    guardar_en_csv(combinacion,accuracy_rf, f1_rf, precision_rf, recall_rf,'resultados_rf.csv')

# Clustering

In [ ]:
X = df.drop('quality', axis=1)
y = df['quality']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

from sklearn.decomposition import PCA

pca = PCA(n_components=3)
X_pca = pca.fit_transform(X_scaled)

pca_df = pd.DataFrame(data=X_pca, columns=['PCA1', 'PCA2', 'PCA3'])
pca_df['quality'] = y

In [ ]:
from plotly import express as px

fig = px.scatter_3d(
    pca_df, x='PCA1', y='PCA2', z='PCA3', 
    color='quality', color_continuous_scale='viridis',
    title='Reducción de Dimensionalidad con PCA (3D)',
    labels={'quality': 'Calidad', 'PCA1': 'Componente 1', 'PCA2': 'Componente 2', 'PCA3': 'Componente 3'}
)

# Mostrar el gráfico interactivo
fig.show()